In [1]:
import pandas as pd
import numpy as np

In [73]:
import gzip, csv
from datetime import datetime
from collections import Counter, defaultdict
from itertools import islice

In [3]:
train = pd.read_csv('../data/train.csv.gz')

In [4]:
df = pd.DataFrame()

In [5]:
def get_dates(d):
    dt = datetime.strptime(d, '%Y-%m-%d %H:%M:%S')
    return dt.hour, dt.month, dt.year

In [6]:
seasons = dict(zip([1,2,3,4,5,6,7,8,9,10,11,12], ['Winter','Winter','Spring','Spring','Spring','Summer','Summer','Summer','Autumn','Autumn','Autumn','Winter']))
def get_season(d):
    return seasons[d]


In [7]:
def get_df(data):
    df = pd.DataFrame()
    df['Hour'], df['Month'], df['Year'] = zip(*data['Dates'].apply(get_dates))
    df['PdDistrict'] = data['PdDistrict']
    if 'Category' in data:
        df['Category'] = data['Category']
    df['Day'] = data['DayOfWeek']
    df['IsPeek'] = df['Hour'].apply(lambda x: 1 if x > 11 and x <= 23 else 0)
    df['Season'] = df['Month'].apply(get_season)
    return df

In [8]:
df = get_df(train)

In [9]:
categories = df['Category'].unique()

In [45]:
def get_group(key):
    values_by_categories = df.groupby('Category')[key].value_counts()
    total_values = df[key].value_counts()
    counts = defaultdict(lambda: {cat: 0 for cat in categories})
    for (cat, item), value in values_by_categories.to_dict().iteritems():
        counts[item][cat] = value / float(total_values[item])
    return counts

In [54]:
percentages_day = get_group('Day')
percentages_hour = get_group('Hour')
percentages_month = get_group('Month')
percentages_year = get_group('Year')
percentages_district = get_group('PdDistrict')
percentages_ispeek = get_group('IsPeek')
percentages_season = get_group('Season')

In [24]:
result = pd.DataFrame(columns=train['Category'].unique())

In [67]:
#test = pd.read_csv('../data/test.csv.gz', nrows=20000)

In [81]:
test_df = get_df(test)

In [102]:
def get_percentages(row):
    c = Counter()
    c.update(percentages_day[row['Day']])
    c.update(percentages_hour[row['Hour']])
    c.update(percentages_month[row['Month']])
    c.update(percentages_year[row['Year']])
    c.update(percentages_district[row['PdDistrict']])
    c.update(percentages_ispeek[row['IsPeek']])
    c.update(percentages_season[row['Season']])
    result = []
    for category in categories:
        result.append(round(c[category] / 7, 6))
    return result

Esto tarda unos minutos

In [103]:
result = [['Id'] + list(categories)]
with gzip.open('../data/test.csv.gz') as f:
    reader = csv.DictReader(f)
    for test in reader:
        row = {'Day': test['DayOfWeek'], 'PdDistrict': test['PdDistrict']}
        row['Hour'], row['Month'], row['Year'] = get_dates(test['Dates'])
        row['IsPeek'] = 1 if row['Hour'] > 11 and row['Hour'] <= 23 else 0
        row['Season'] = get_season(row['Month'])
        result.append([test['Id']] + get_percentages(row))

In [111]:
len(result)

884263

In [110]:
with gzip.open('nav2.csv.gz', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for row in result:
        writer.writerow(row)